# Module 2.A - Where Data Comes From (Real-World Pipeline Starter)

### Core dataset for the whole module 

**NYC 311 Service Requests (2020 - present)** (NYC Open Data/Socrata)  
Why this dataset works for learning:
* **Real mess:** missing values, inconsistent strings, free-text fields, and "weird" categories.
* **Real scale:** the full dataset is huge, so must learn how to pull a *slice*
* **Multiple access modes:** the same data is available as **CSV, API (JSON), SQL**
* **Real change over time:** fields and value distributions can shift (schema drift)

Will also create two supporting assets that will be reused later:
* **Data dictionary Excel file:** (`.xlsx`) from the publisher (documentation)
* **Borough reference table:** either scraped from the web or created as a seed file

### What will be built in 2.A  

Will produce a local, module-scoped workspace. Organized by module and stored as described:  
```bash
~/work/m2/data/
    raw/          # downloaded files, API responses
    reference/    # lookup tables, dictionaries
    warehouse/    # SQLite databases
```

Later notebooks will assume these exist:
* **2.B** Data quality: missingness, duplicates, inconsistent categories, schema drift
* **2.C** Wrangling: groupby, joins, string cleaning, feature construction
* **2.D** Scaling: incremental refresh, "raw &rarr; staged &rarr; curated" thinking
* **2.E** Outliers/validation: response times, anomaly checks, "is this plausible?" rules

The goal in 2.A is not perfect cleaning. It is learning how to acquire data reliably and keep the process reproducible

## Setup (requests, BeautifulSoup, and a writable workspace)

Common libraries:
* **requests:** a simple way to make HTTP requests
* **BeautifulSoup:** parses HTML to extract pieces

In [2]:
from __future__ import annotations

import json
import sqlite3
from datetime import datetime, timedelta, timezone
from pathlib import Path
from typing import Any, Dict, List

import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

# Writable workspace (module-scoped)
WORK_DIR   = Path("work")
MODULE_DIR = WORK_DIR / "m2"
DATA_DIR   = MODULE_DIR / "data"
RAW_DIR    = DATA_DIR / "raw"
REF_DIR    = DATA_DIR / "reference"
WH_DIR     = DATA_DIR / "warehouse"

for d in [RAW_DIR, REF_DIR, WH_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Writable Module 2 data workspace ready:")
print(" ", DATA_DIR.resolve())

Writable Module 2 data workspace ready:
  /home/glake/Nextcloud/Classwork/CS6678 - Advanced Machine Learning/Jupyter Notebooks/work/m2/data


## A.0 - Source Audit Template

Before cleaning, provide answers to:  
* What does one row represent
* What system produced it?
* What time range does it cover?
* What are known limitations?
* Which fields look risky (missing, free-text, inconsistent categories)?

We will keep a small structured dictionary of notes that can be reused later

In [3]:
source_audit = {
    "dataset_name": "NYC 311 Service Requests (2020-present)",
    "publisher": "NYC Open Data / 311",
    "where_it_comes_from": "City 311 request intake system (customer service requests routed to agencies).",
    "unit_of_analysis": "Each row represents one 311 service request.", 
    "time_grain": "Requests are created continuously; rows include timestamps for created/closed when available.",
    "known_limitations": [
        "Many fields are optional depending on request type (expect missingness).",
        "Free-text fields (descriptor/address) can be inconsistent and messy.",
        "The dataset is continuously updated; results can change between runs.",
    ],
    "Notes 1/30/26": []
}

source_audit

{'dataset_name': 'NYC 311 Service Requests (2020-present)',
 'publisher': 'NYC Open Data / 311',
 'where_it_comes_from': 'City 311 request intake system (customer service requests routed to agencies).',
 'unit_of_analysis': 'Each row represents one 311 service request.',
 'time_grain': 'Requests are created continuously; rows include timestamps for created/closed when available.',
 'known_limitations': ['Many fields are optional depending on request type (expect missingness).',
  'Free-text fields (descriptor/address) can be inconsistent and messy.',
  'The dataset is continuously updated; results can change between runs.'],
 'Notes 1/30/26': []}

## A.1 Files (CSV): Download a Reproducible Slice

Large public datasets are often too big to download in full for learning. A useful technique is to define a slice that is:  
* small enough to iterate quickly (seconds, not minutes)
* recent enough to include real mess
* refreshable

We will pull the **last 14 days** of NYC 311 requests as a CSV  

**Note on Socrata Timestamps**  

NYC Open Data uses Socrata. Many timestamp fields are "floating timestamps" and expect ISO8601 without timezone suffixes (No Z, No +00:00) in the query string. So we format timestamps like: `2026-01-04T04:03:21`

In [4]:
NYC311_BASE = "https://data.cityofnewyork.us/resource/erm2-nwe9"

# Stable subset of columns that will be reused across Module 2.
NYC311_COLUMNS = [
    "unique_key",
    "created_date",
    "closed_date",
    "agency",
    "agency_name",
    "complaint_type",
    "descriptor",
    "status",
    "borough",
    "incident_zip",
    "incident_address",
    "street_name",
    "city",
    "latitude",
    "longitude",
]

def iso_floating(dt: datetime) -> str:
    """
    Socrata floating timestamps expect ISO8601 without timezone suffix.
    We will drop tzinfo and milliseconds to be conservative
    """
    dt = dt.astimezone(timezone.utc).replace(tzinfo=None)
    return dt.strftime("%Y-%m-%dT%H:%M:%S")

def download_if_missing(url: str, path: Path, params: dict | None = None, timeout: int = 30) -> Path:
    """
    Download a URL to disk only if the file is not already cached.
    We chace downloads so later notebooks (2.B - 2.E) can reuse the same files
    without hammering the public API repeatedly.
    """
    if path.exists() and path.stat().st_size > 0:
        print("Used cached:", path)
        return path

    print("Downloading:", url)
    r = requests.get(url, params=params, timeout=timeout)

    if r.status_code >= 400:
        print("Status:", r.status_code)
        print("Body (first 300 chars):", r.text[:300])

    r.raise_for_status()
    path.write_bytes(r.content)
    print("Saved:", path, f"({path.stat().st_size/1e6:.2f} MB)")
    return path

def socrata_csv_params(days: int=14, limit: int=5000) -> dict:
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=days)

    select = ",".join(NYC311_COLUMNS)
    where = (
        f"created_date >= '{iso_floating(start)}'"
        f"AND created_date < '{iso_floating(end)}'"
    )
    return {"$select": select, "$where": where, "$order": "created_date DESC", "$limit": limit}

CSV_PATH = RAW_DIR / "nyc311_last14d.csv"
params = socrata_csv_params(days=14, limit=50000)

print("Where clause:", params["$where"])
download_if_missing(f"{NYC311_BASE}.csv", CSV_PATH, params=params)

Where clause: created_date >= '2026-01-17T14:18:35'AND created_date < '2026-01-31T14:18:35'
Used cached: work/m2/data/raw/nyc311_last14d.csv


PosixPath('work/m2/data/raw/nyc311_last14d.csv')

### Load the CSV and do a quick source audit

In [5]:
df_csv = pd.read_csv(CSV_PATH)
df_csv.head(5)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,NaN,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30T01:50:53.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30T01:50:52.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30T01:50:32.000,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


In [6]:
df_csv.dtypes

unique_key            int64
created_date         object
closed_date          object
agency               object
agency_name          object
complaint_type       object
descriptor           object
status               object
borough              object
incident_zip        float64
incident_address     object
street_name          object
city                 object
latitude            float64
longitude           float64
dtype: object

In [7]:
df_csv.isna().mean().sort_values(ascending=False).head(10)

closed_date         0.60766
city                0.07640
street_name         0.02592
incident_address    0.02588
latitude            0.01222
longitude           0.01222
incident_zip        0.00732
descriptor          0.00680
complaint_type      0.00000
agency              0.00000
dtype: float64

**Update Source Audit**  

Add 3 observations to `source_audit["today_notes"]`

In [11]:
# clear current notes so that the cell does not create duplicate entries
source_audit["Notes 1/30/26"] = []
source_audit["Notes 1/30/26"].append("incident_zip does not need to be a float64")
source_audit["Notes 1/30/26"].append("There is at least one entry that is incomplete")
source_audit["Notes 1/30/26"].append("closed_date is often missing")

source_audit

{'dataset_name': 'NYC 311 Service Requests (2020-present)',
 'publisher': 'NYC Open Data / 311',
 'where_it_comes_from': 'City 311 request intake system (customer service requests routed to agencies).',
 'unit_of_analysis': 'Each row represents one 311 service request.',
 'time_grain': 'Requests are created continuously; rows include timestamps for created/closed when available.',
 'known_limitations': ['Many fields are optional depending on request type (expect missingness).',
  'Free-text fields (descriptor/address) can be inconsistent and messy.',
  'The dataset is continuously updated; results can change between runs.'],
 'Notes 1/30/26': ['incident_zip does not need to be a float64',
  'There is at least one entry that is incomplete',
  'closed_date is often missing']}

## A.2 SQL databases (SQLite): Land Raw Data Into a Local Warehouse

In [12]:
DB_PATH = WH_DIR / "module2.db"
conn = sqlite3.connect(DB_PATH)
print("DB:", DB_PATH.resolve())

DB: /home/glake/Nextcloud/Classwork/CS6678 - Advanced Machine Learning/Jupyter Notebooks/work/m2/data/warehouse/module2.db


### Write the raw CSV into a table

In [13]:
df_csv.to_sql("nyc311_raw", conn, if_exists="replace", index=False)

cur = conn.cursor()
cur.execute("CREATE INDEX IF NOT EXISTS idx_nyc311_created_date ON nyc311_raw(created_date)")
cur.execute("CREATE INDEX IF NOT EXISTS idx_nyc311_borough ON nyc311_raw(borough)")
cur.execute("CREATE INDEX IF NOT EXISTS idx_nyc311_complaint_type ON nyc311_raw(complaint_type)")
conn.commit()

print("Rows in nyc311_raw:", cur.execute("SELECT COUNT(*) FROM nyc311_raw").fetchone()[0])

Rows in nyc311_raw: 50000


### SQL sanity checks

In [14]:
q = """
SELECT borough, COUNT(*) AS n
FROM nyc311_raw
GROUP BY borough
ORDER BY n DESC
"""

pd.read_sql_query(q, conn).head(20)

,borough,n
0,BROOKLYN,15654
1,QUEENS,11831
2,BRONX,10806
3,MANHATTAN,8876
4,STATEN ISLAND,2798
5,Unspecified,35


In [15]:
q = """
SELECT complaint_type, COUNT(*) AS n
FROM nyc311_raw
GROUP BY complaint_type
ORDER BY n DESC
LIMIT 15
"""

pd.read_sql_query(q, conn)

,complaint_type,n
0,Snow or Ice,12376
1,HEAT/HOT WATER,11890
2,Illegal Parking,4922
3,Blocked Driveway,2790
4,Noise - Residential,2492
5,UNSANITARY CONDITION,1288
6,PLUMBING,1212
7,PAINT/PLASTER,819
8,DOOR/WINDOW,773
9,WATER LEAK,750


## A.3 APIs: query the same dataset via Socrata (JSON)

In [16]:
def socrata_json(days: int=14, limit: int=1000, offset: int=0) -> List[Dict[str, Any]]:
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=days)

    select = ",".join(NYC311_COLUMNS)
    where = (
        f"created_date >= '{iso_floating(start)}'"
        f"AND created_date < '{iso_floating(end)}'"
    )

    params = {"$select": select, "$where": where, "$order": "created_date DESC", "$limit": limit, "$offset": offset}
    r = requests.get(f"{NYC311_BASE}.json", params=params, timeout=30)

    if r.status_code >= 400:
        print("Status:", r.status_code)
        print("Where:", where)
        print("Body (first 300 chars):", r.text[300])
    r.raise_for_status()
    return r.json()

page1 = socrata_json(days=14, limit=1000, offset=0)
page2 = socrata_json(days=14, limit=1000, offset=1000)
records = page1 + page2

print("Records fetched (JSON):", len(records))
print("Example record keys:", list(records[0].keys())[:12])

Records fetched (JSON): 2000
Example record keys: ['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type', 'descriptor', 'status', 'borough']


### Save raw JSON

In [17]:
JSON_PATH = RAW_DIR / "nyc311_last14d.json"
if not JSON_PATH.exists():
    JSON_PATH.write_text(json.dumps(records, indent=2))
    print("Saved:", JSON_PATH)
else:
    print("Using cached:", JSON_PATH)

df_api = pd.json_normalize(records)
df_api.head(3)

Saved: work/m2/data/raw/nyc311_last14d.json


,unique_key,created_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude,closed_date
0,67720523,2026-01-30T01:51:21.000,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.64978696357874,-73.95854980692795,NaN
2,67758820,2026-01-30T01:50:53.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.80049840941944,-73.96567975561733,NaN


### Compare CSV vs API quickly

In [18]:
comparison_notes = {
    "csv_columns": len(df_csv.columns),
    "api_columns": len(df_api.columns),
    "columns_only_in_csv": sorted(set(df_csv.columns) - set(df_api.columns))[:20],
    "columns_only_in_api": sorted(set(df_api.columns) - set(df_csv.columns))[:20],
    "csv_missing_rate_top10": df_csv.isna().mean().sort_values(ascending=False).head(10).to_dict(),
    "api_missing_rate_top10": df_api.isna().mean().sort_values(ascending=False).head(10).to_dict(),
}

comparison_notes


{'csv_columns': 15,
 'api_columns': 15,
 'columns_only_in_csv': [],
 'columns_only_in_api': [],
 'csv_missing_rate_top10': {'closed_date': 0.60766,
  'city': 0.0764,
  'street_name': 0.02592,
  'incident_address': 0.02588,
  'latitude': 0.01222,
  'longitude': 0.01222,
  'incident_zip': 0.00732,
  'descriptor': 0.0068,
  'complaint_type': 0.0,
  'agency': 0.0},
 'api_missing_rate_top10': {'closed_date': 0.649,
  'city': 0.046,
  'street_name': 0.024,
  'incident_address': 0.024,
  'latitude': 0.019,
  'longitude': 0.019,
  'incident_zip': 0.0075,
  'status': 0.0,
  'descriptor': 0.0,
  'complaint_type': 0.0}}

## A.4 Reference Data: Borough Lookup (Scrape OR Seed File)

Try **Scrape** first, if it fails (403), fall back to **Seed File**

In [19]:
BORO_PATH = REF_DIR / "nyc_boroughs.csv"

if BORO_PATH.exists():
    print("Using cached:", BORO_PATH)
    df_boro = pd.read_csv(BORO_PATH)
    display(df_boro)
else:
    WIKI_URL = "https://en.wikipedia.org/wiki/Boroughs_of_New_York_City"
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (educational; JupyterLab) requests",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
    }

    try:
        r = requests.get(WIKI_URL, headers=HEADERS, timeout=30)
        if r.status_code >= 400:
            raise RuntimeError(f"HTTP {r.status_code} from Wikipedia")

        soup = BeautifulSoup(r.text, "html.parser")

        tables = soup.find_all("table", class_="wikitable")
        target = None
        for t in tables:
            header_text = t.get_text(" ", strip=True).lower()
            if "borough" in header_text and "population" in header_text:
                target = t
                break

        if target is None:
            raise RuntimeError("Could not find expected borough table (page structure may have changed).")

        rows = []
        for tr in target.find_all("tr")[1:]:
            tds = tr.find_all(["th", "td"])
            if not tds:
                continue
            borough = tds[0].get_text(" ", strip=True).strip()
            if borough and borough.lower() != "borough":
                rows.append({"borough": borough})

        df_boro = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)
        df_boro.to_csv(BORO_PATH, index=False)
        print("Option A success: scraped boroughs and saved:", BORO_PATH)
    
    except Exception as e:
        print("Option A failed (scraping). Reason:", e)
        print("Falling back to Option B (seed file).")
    
        df_boro = pd.DataFrame([
            {"borough": "BRONX"},
            {"borough": "BROOKLYN"},
            {"borough": "MANHATTAN"},
            {"borough": "QUEENS"},
            {"borough": "STATEN ISLAND"},
        ])
        df_boro.to_csv(BORO_PATH, index=False)
        print("Option B success: saved seed borough table:", BORO_PATH)
    
    display(df_boro)

Option A success: scraped boroughs and saved: work/m2/data/reference/nyc_boroughs.csv


,borough
0,Jurisdiction
1,The Bronx
2,Brooklyn
3,Manhattan
4,Queens
5,Staten Island
6,City of New York
7,State of New York
8,Sources : [ 3 ] [ 4 ] [ 5 ] [ 6 ] and see indi...


**Notes:**  
* Scraping can break even if the code is correct (403, HTML changes)
* In many teams, seed files are the standard approach for small reference tables.

## A.5 Excel: Download the 311 data dictionary (XLSX)

In [20]:
DATA_DICTIONARY_URL = (
    "https://nycopendata.socrata.com/api/views/erm2-nwe9/files/"
    "b372b884-f86a-453b-ba16-1fe06ce9d212?download=true&filename=311_ServiceRequest_2010-Present_DataDictionary_Updated_2023.xlsx"
)

XLSX_PATH = REF_DIR / "nyc311_data_dictionary.xlsx"
download_if_missing(DATA_DICTIONARY_URL, XLSX_PATH)

Downloading: https://nycopendata.socrata.com/api/views/erm2-nwe9/files/b372b884-f86a-453b-ba16-1fe06ce9d212?download=true&filename=311_ServiceRequest_2010-Present_DataDictionary_Updated_2023.xlsx
Saved: work/m2/data/reference/nyc311_data_dictionary.xlsx (0.50 MB)


PosixPath('work/m2/data/reference/nyc311_data_dictionary.xlsx')

In [22]:
import openpyxl
wb = openpyxl.load_workbook(XLSX_PATH, read_only=True)
wb.sheetnames

['Dataset Information',
 'Column Information',
 'All Agencies Complaint<>Details',
 'HPD Complaint<>Details',
 'Dataset Revision History',
 'Primer Page & InternaI Informat',
 'Hidden_Frequencies',
 'Hidden_Agencies',
 'Hidden_DataTypes']

In [23]:
sheet = wb.sheetnames[0]
df_dict_preview = pd.read_excel(XLSX_PATH, sheet_name=sheet)
df_dict_preview.head(20)

/home/glake/miniconda3/envs/ml/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


,Unnamed: 0,Unnamed: 1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,Data Dictionary - Dataset Information,NaN
7,Dataset Name,311 Service Requests from 2010 to Present
8,Dataset URL,https://data.cityofnewyork.us/Social-Services/...
9,Data Provided by\nThe name of the NYC agency p...,311


Many data dictionaries are human-formatted spreadsheets with title blocks and notes. The first read may look messy (Unnamed columns, NaNs). That is normal

## A.6 Wrap-up: Verify Reusable Artifacts Exist

In [24]:
paths = [CSV_PATH, JSON_PATH, BORO_PATH, XLSX_PATH, DB_PATH]
[(str(p), p.exists(), round(p.stat().st_size/1e6, 3) if p.exists() else None) for p in paths]

[('work/m2/data/raw/nyc311_last14d.csv', True, 12.143),
 ('work/m2/data/raw/nyc311_last14d.json', True, 1.036),
 ('work/m2/data/reference/nyc_boroughs.csv', True, 0.0),
 ('work/m2/data/reference/nyc311_data_dictionary.xlsx', True, 0.497),
 ('work/m2/data/warehouse/module2.db', True, 13.627)]

In [25]:
reflection = {
    "row_definition": "Each row represents one 311 service request.",
    "incremental_key": "",
    "high_risk_columns": [],
    "sensitive_columns": [],
}
reflection


{'row_definition': 'Each row represents one 311 service request.',
 'incremental_key': '',
 'high_risk_columns': [],
 'sensitive_columns': []}

# Module 2B - Data Quality & Structure (NYC 311)

in this notebook we will practice a realistic workflow.  
1. **Load** the same NYC311 slice created in 2.A
2. **Audit quality:** missingness, duplicates, inconsistencies
3. **Detect drift:** "today vs yesterday" schema differences
4. Produce a short **Data Quality Report** that will be reused in later modules

**Note**:  
We are not trying to make the dataset perfect.  
We are trying to make it **trustworthy enough for a special decision** and to document what was done.

## 2B.0 - Setup

This notebook expects that **Module 2.A** has already been ran and that a module data workspace has been created  

`/work/m2/data/`

In [35]:
import numpy as np
import pandas as pd

## 2B.1 - Load the Data and Establish a Row Definition

**Row definition (target):** Each row represents one 311 service request, identified by `unique_key`  

Before cleaning, we load as-is and profile the mess

In [27]:
df_raw = pd.read_csv(CSV_PATH)
print("Shape:", df_raw.shape)
df_raw.head(5)

Shape: (50000, 15)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
0,67720523,2026-01-30T01:51:21.000,NaN,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Unspecified,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,67746090,2026-01-30T01:51:04.000,NaN,DOE,Department of Education,School Maintenance,Heating Problem,In Progress,BROOKLYN,11226.0,911 FLATBUSH AVENUE,FLATBUSH AVENUE,BROOKLYN,40.649787,-73.958550
2,67758820,2026-01-30T01:50:53.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,MANHATTAN,10025.0,936 AMSTERDAM AVENUE,AMSTERDAM AVENUE,NEW YORK,40.800498,-73.965680
3,67707975,2026-01-30T01:50:52.000,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,In Progress,STATEN ISLAND,10302.0,190 TRANTOR PLACE,TRANTOR PLACE,STATEN ISLAND,40.629156,-74.144411
4,67771794,2026-01-30T01:50:32.000,NaN,TLC,Taxi and Limousine Commission,Taxi Complaint,Driver Complaint - Non Passenger,In Progress,QUEENS,11430.0,JOHN F KENNEDY AIRPORT,JOHN F KENNEDY AIRPORT,JAMAICA,40.648320,-73.788281


In [28]:
df_raw.dtypes.to_frame("dtype").T

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,status,borough,incident_zip,incident_address,street_name,city,latitude,longitude
dtype,int64,object,object,object,object,object,object,object,object,float64,object,object,object,float64,float64


### Create quality notes  

Write row definition and first impressions. Add 3-6 bullets:
* What does one row represent?
* What columns look risky?
* What looks like "encoded missingness"?

In [31]:
quality_notes = {
    "row_definition": "Each row is one 311 service request (unique_key) created at created_date.",
    "data_type_notes": [
        "incident_zip should be treated as a string to preserve leading zeros.",
        "descriptor/incident_address are free-text and likely inconsistent.",
    ],
    "completeness_issues": [],
    "date_added": "1/30/26"
}
quality_notes

{'row_definition': 'Each row is one 311 service request (unique_key) created at created_date.',
 'data_type_notes': ['incident_zip should be treated as a string to preserve leading zeros.',
  'descriptor/incident_address are free-text and likely inconsistent.'],
 'completeness_issues': [],
 'date_added': '1/30/26'}

## 2B.2 - Missing data: Measure, then explain

First rule: **do not impute yet**  
Start by quantifying missingness and asking why it might be missing

In [32]:
missing_rate = df_raw.isna().mean().sort_values(ascending=False)
missing_rate.head(15)

closed_date         0.60766
city                0.07640
street_name         0.02592
incident_address    0.02588
latitude            0.01222
longitude           0.01222
incident_zip        0.00732
descriptor          0.00680
complaint_type      0.00000
agency              0.00000
created_date        0.00000
unique_key          0.00000
agency_name         0.00000
status              0.00000
borough             0.00000
dtype: float64

### Normalize common "missing" encodings  

Many real datasets use sentinel values like `""`, `"UNKNOWN"`, `"N/A"`, or whitespace  

Create a `df` as a cleaned copy while keeping `df_raw` unchanged.

In [36]:
df = df_raw.copy()

MISSING_TOKENS = {"", " ", "  ", "UNKNOWN", "Unknown", "N/A", "NA", "NULL", "null"}

for col in df.columns:
    if df[col].dtype == "object":
        # Normalize whitespace-only strings
        df[col] = df[col].astype(str).str.strip()
        df.loc[df[col].isin(MISSING_TOKENS), col] = np.nan

# Parse timestamps (coerce errors to NaT)
for col in ["created_date", "closed_date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Treat ZIP as string (preserve leading zeros, handle NaN)
if "incident_zip" in df.columns:
    df["incident_zip"] = df["incident_zip"].astype("string")

df.isna().mean().sort_values(ascending=False).head(15)

closed_date         0.60766
longitude           0.01222
latitude            0.01222
incident_zip        0.00732
agency              0.00000
unique_key          0.00000
created_date        0.00000
descriptor          0.00000
complaint_type      0.00000
agency_name         0.00000
status              0.00000
incident_address    0.00000
borough             0.00000
city                0.00000
street_name         0.00000
dtype: float64

### Missingness by group  

If missingness varies a lot by group (borough, agency, complaint type), it is often not random, and that affects modeling and fairness  

In [37]:
# Choose a column with substantial missingness to inspect
target_col = "incident_address" if "incident_address" in df.columns else df.columns[df.isna().mean().idxmax()]
print("Inspecting missingness for:", target_col)

group_col = "borough" if "borough" in df.columns else "agency"
tmp = df[[group_col, target_col]].copy()
tmp["is_missing"] = tmp[target_col].isna().astype(int)

missing_by_group = tmp.groupby(group_col)["is_missing"].mean().sort_values(ascending=False)
missing_by_group.head(10)

Inspecting missingness for: incident_address


borough
BRONX            0.0
BROOKLYN         0.0
MANHATTAN        0.0
QUEENS           0.0
STATEN ISLAND    0.0
Unspecified      0.0
Name: is_missing, dtype: float64

In [38]:
# Check another column - Closed Date
target_col = "closed_date" if "closed_date" in df.columns else df.columns[df.isna().mean().idxmax()]
print("Closed Date missingness for:", target_col)

group_col = "borough" if "borough" in df.columns else "agency"
tmp = df[[group_col, target_col]].copy()
tmp["is_missing"] = tmp[target_col].isna().astype(int)

missing_by_group = tmp.groupby(group_col)["is_missing"].mean().sort_values(ascending=False)
missing_by_group.head(10)


Closed Date missingness for: closed_date


borough
Unspecified      0.857143
STATEN ISLAND    0.728735
MANHATTAN        0.631478
BROOKLYN         0.598952
BRONX            0.593374
QUEENS           0.584989
Name: is_missing, dtype: float64

### Task - Missingness Hypothesis  

Pick one column with missingness > 20% and answer:
1. What might cause it to be missing?
2. Is it "optional by design" (depends on complaint type)?
3. Would dropping rows with missing values change what the data represents?

In [41]:
missingness_hypothesis = {
    "Column: closed_date" : "often missing because requests are still open, or closure not required",
    "More data" : "More data"
}

missingness_hypothesis

{'Column: closed_date': 'often missing because requests are still open, or closure not required',
 'More data': 'More data'}

## 2B.3 - Duplicates: Exact vs Entity Duplicates

For this dataset, `unique_key` should identify a request.  
* **Exact duplicates:** identical rows repeated (export/pipeline issues)
* **Key duplicates:** `unique_key` repeats (should be rare; indicates ingestion error)

In [42]:
# Exact duplicates across all columns
exact_dup_count = df.duplicated().sum()
print("Exact duplicate rows:", int(exact_dup_count))

# Duplicates in the request identifier
if "unique_key" in df.columns:
    key_dup_count = df["unique_key"].duplicated().sum()
    print("Duplicate unique_key values:", int(key_dup_count))
else:
    print("No unique_key column found.")

Exact duplicate rows: 0
Duplicate unique_key values: 0


In [43]:
if "unique_key" in df.columns and df["unique_key"].duplicated().any():
    dup_keys = df.loc[df["unique_key"].duplicated(keep=False), "unique_key"].head(10).tolist()
    df[df["unique_key"].isin(dup_keys)].sort_values("unique_key").head(20)
else:
    print("No key duplicates found (good).")

No key duplicates found (good).


### Decide a depupe rule (if needed)

If key duplicates exists, must choose a rule:
* keep the newest record (by created_date)
* keep the most complete record
* keep the first seen (not recommended unless you know ordering is stable)

In [44]:
def dedupe_by_key_keep_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    """Example rule: keep the row with the fewest missing values per key."""
    if key not in df_in.columns:
        return df_in.copy()

    df_tmp = df_in.copy()
    miss = df_tmp.isna().sum(axis=1)
    df_tmp["_missing_count"] = miss
    df_tmp = df_tmp.sort_values([key, "_missing_count"]).drop_duplicates(subset=[key], keep="first")
    return df_tmp.drop(columns=["_missing_count"])

df_deduped = dedupe_by_key_keep_most_complete(df, "unique_key")
print("Before:", df.shape, "| After:", df_deduped.shape)

Before: (50000, 15) | After: (50000, 15)


**Dedupe Notes:**  

If duplicates are present, document the rule. If none are present document what was checked.

In [45]:
dedupe_notes = {
    "exact_duplicates": "None found (or dropped them).",
    "unique_key_duplicates": "None found (or deduped by most complete row).",
    "rationale": "unique_key should be the request identifier; duplicates would indicate pipeline/export issues.",
    "deduplication_date": "1/30/26"
}
dedupe_notes

{'exact_duplicates': 'None found (or dropped them).',
 'unique_key_duplicates': 'None found (or deduped by most complete row).',
 'rationale': 'unique_key should be the request identifier; duplicates would indicate pipeline/export issues.',
 'deduplication_date': '1/30/26'}